<a href="https://colab.research.google.com/github/SIDIBEMoussa/Texte_mining-and-web-mining/blob/main/Text_mining_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

link to dataset:
https://www.kaggle.com/bittlingmayer/amazonreviews?select=test.ft.txt.bz2

Chargement des libraries

In [2]:
!pip install fasttext 

In [3]:
import fasttext
import pandas as pd
import numpy as np
import bz2
import csv
from sklearn.metrics import roc_auc_score
import os
from google.colab import drive

Connection à drive où son stockés les données de l'étude

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Chargement des données d'études

In [5]:
data=bz2.BZ2File("/content/drive/MyDrive/Text Classification/train.ft.txt.bz2")
data=data.readlines()
data=[x.decode('utf-8') for x in data]
print(len(data))

3600000


In [6]:
data[0:5]

['__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^\n',
 "__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.\n",
 '__label__2 Amazing!: This s

In [7]:
data=pd.DataFrame(data)
data.to_csv("/content/drive/MyDrive/Text Classification/train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar=" ", escapechar=" ")

In [8]:
model = fasttext.train_supervised('/content/drive/MyDrive/Text Classification/train.txt',label_prefix='__label__', thread=4, epoch = 10)
print(model.labels, 'are the labels or targets the model is predicting')

['__label__1', '__label__2'] are the labels or targets the model is predicting


In [9]:
test = bz2.BZ2File("/content/drive/MyDrive/Text Classification/test.ft.txt.bz2")
test = test.readlines()
test = [x.decode('utf-8') for x in test]
print(len(test), 'number of records in the test set') 

# To run the predict function, we need to remove the __label__1 and __label__2 from the testset.  
new = [w.replace('__label__2 ', '') for w in test]
new = [w.replace('__label__1 ', '') for w in new]
new = [w.replace('\n', '') for w in new]

# Use the predict function 
pred = model.predict(new)

# check the first record outputs
print(pred[0][0], 'is the predicted label')
print(pred[0][1], 'is the probability score')

400000 number of records in the test set
['__label__2'] is the predicted label
['__label__2'] is the probability score


In [10]:
# Lets recode the actual targets to 1's and 0's from both the test set and the actual predictions  
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

# run the accuracy measure. 
print(roc_auc_score(labels, pred_labels))

0.9173499999999999


In [11]:
data=bz2.BZ2File("/content/drive/MyDrive/Text Classification/train.ft.txt.bz2")
data=data.readlines()
data=[x.decode('utf-8') for x in data]
print(len(data))

3600000


In [12]:
data=pd.DataFrame(data)
data.head()
train_size=int(0.70*len(data))
train_size

2520000

In [13]:
data[:train_size].to_csv("/content/drive/MyDrive/Text Classification/train_op.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar=" ", escapechar=" ")
data[train_size:].to_csv("/content/drive/MyDrive/Text Classification/validation_op.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar=" ", escapechar=" ")

In [14]:
model1 = fasttext.train_supervised(
    input='/content/drive/MyDrive/Text Classification/train_op.txt',
    autotuneValidationFile='/content/drive/MyDrive/Text Classification/validation_op.txt',
    label_prefix='__label__',
     thread=4, epoch = 10)
print(model1.labels, 'are the labels or targets the model is predicting')

RuntimeError: ignored

In [ ]:
# Use the predict function 
pred = model1.predict(new)

In [ ]:
# Lets recode the actual targets to 1's and 0's from both the test set and the actual predictions  
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

# run the accuracy measure. 
print(roc_auc_score(labels, pred_labels))